# 03 - 格式化為訓練資料（SFT JSONL / Chat messages + Schema 驗證）

目標：
- 載入 `outputs/clean.jsonl`
- 轉換為常見訓練格式：
  - **SFT JSONL**：`{"messages": {"human": "...", "gpt": "..."}}`
  - **Chat messages**：保留 `{role, content}` 陣列
- 使用 **JSON Schema** 驗證結構正確性
- 輸出 `outputs/sft.jsonl` 與（或）`outputs/messages.jsonl`

## 1.安裝依賴與路徑設定

In [1]:
!pip -q install jsonschema pandas

In [2]:
from pathlib import Path
import json
import pandas as pd
from typing import List, Dict, Any

INPUT_CLEAN = Path("outputs/clean.jsonl")
OUTPUT_DIR = Path("outputs")
OUTPUT_SFT = OUTPUT_DIR / "sft.jsonl"
OUTPUT_MSG = OUTPUT_DIR / "messages.jsonl"

assert INPUT_CLEAN.exists(), f"找不到 {INPUT_CLEAN}，請先完成 02_quality_checks.ipynb"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 讀取：", INPUT_CLEAN)
print("📝 輸出（SFT）：", OUTPUT_SFT)
print("📝 輸出（messages）：", OUTPUT_MSG)

✅ 讀取： outputs/clean.jsonl
📝 輸出（SFT）： outputs/sft.jsonl
📝 輸出（messages）： outputs/messages.jsonl


## 2. 載入 `clean.jsonl`
保留原始欄位（`id`, `topic`, `style`, `messages`），後續轉換時會使用。

In [3]:
records: List[Dict[str, Any]] = []
with INPUT_CLEAN.open("r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

len(records), records[0].keys()

(25, dict_keys(['id', 'topic', 'style', 'messages']))

## 3. 兩種常見訓練格式

### A. Alpca Format